In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# device config
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# hyperparameters
num_epochs = 4
batch_size = 4
lr = 0.001

In [3]:
transform = transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5),
                                                   (0.5, 0.5, 0.5))])

In [4]:
# dataset
train_dataset = torchvision.datasets.CIFAR10(root = "data/",
                                          train = True,
                                          transform = transform,
                                          download = True)

test_dataset = torchvision.datasets.CIFAR10(root = "data/",
                                          train = False,
                                          transform = transform,
                                          download = True)

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                          batch_size = batch_size,
                                          shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = batch_size,
                                          shuffle = False)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data/
Files already downloaded and verified


In [5]:
classes = ("plane", "car", "bird", "cat", "deer",
          "dog", "frog", "horse", "ship", "truck")

In [8]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6 ,5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [9]:
model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = lr)

In [10]:
# training loop
n_total_steps = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # reshape
        images = images.to(device)
        labels = labels.to(device)
        
        # forward
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # backwards
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 2000 == 0:
            print("epoch {}/{}, step {}/{}, loss = {:.4f}".format(epoch+1,
                                                                 num_epochs,
                                                                 i+1,
                                                                 n_total_steps,
                                                                 loss))

epoch 1/4, step 2000/12500, loss = 2.3200
epoch 1/4, step 4000/12500, loss = 2.3060
epoch 1/4, step 6000/12500, loss = 2.2796
epoch 1/4, step 8000/12500, loss = 2.2903
epoch 1/4, step 10000/12500, loss = 2.3081
epoch 1/4, step 12000/12500, loss = 2.2959
epoch 2/4, step 2000/12500, loss = 2.1028
epoch 2/4, step 4000/12500, loss = 2.6593
epoch 2/4, step 6000/12500, loss = 2.3522
epoch 2/4, step 8000/12500, loss = 1.8835
epoch 2/4, step 10000/12500, loss = 2.0662
epoch 2/4, step 12000/12500, loss = 1.7459
epoch 3/4, step 2000/12500, loss = 1.7317
epoch 3/4, step 4000/12500, loss = 2.4506
epoch 3/4, step 6000/12500, loss = 2.6507
epoch 3/4, step 8000/12500, loss = 2.7514
epoch 3/4, step 10000/12500, loss = 1.8778
epoch 3/4, step 12000/12500, loss = 2.4160
epoch 4/4, step 2000/12500, loss = 1.2717
epoch 4/4, step 4000/12500, loss = 1.1909
epoch 4/4, step 6000/12500, loss = 1.3429
epoch 4/4, step 8000/12500, loss = 1.1353
epoch 4/4, step 10000/12500, loss = 1.0551
epoch 4/4, step 12000/12500

In [12]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device) 
        outputs = model(images)
        
        # value, index
        _, predictions = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predictions == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predictions[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1
    
    acc = 100.0 * n_correct / n_samples
    print("Accuracy of the network = {}".format(acc))
    
    for i in range(10):
        class_acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print("Accuracy of {} class = {}".format(classes[i], class_acc))

Accuracy of the network = 43.94
Accuracy of plane class = 63.0
Accuracy of car class = 54.9
Accuracy of bird class = 17.9
Accuracy of cat class = 33.8
Accuracy of deer class = 34.0
Accuracy of dog class = 43.9
Accuracy of frog class = 49.7
Accuracy of horse class = 53.6
Accuracy of ship class = 37.9
Accuracy of truck class = 50.7
